# Knapsack

In [2]:
# imports
import numpy as np
from fractions import Fraction
np.set_printoptions(formatter={'object':lambda x: str(Fraction(x).limit_denominator())})

In [3]:
# initialize variables

# a: volume
# b: limit
# c: cost

# a = np.array([17,16,21,8,12]) + Fraction()
# b = 37
# c = np.array([11,14,16,8,7], ) + Fraction()

# a = np.array([10,7,5,4]) + Fraction()
# b = 20
# c = np.array([25,21,30,8], ) + Fraction()

a = np.array([29,12,36,24]) + Fraction()
b = 73
c = np.array([87,17,72,96], ) + Fraction()

c_a = c/a
J = np.arange(len(a), dtype=int)
J_prime = c_a.argsort()[::-1]

a = a[J_prime]
c = c[J_prime]
c_a = c_a[J_prime]

print(f"{'old objects j`':14s} | ", end = "")
[print(f"{str(j+1):6s}  ", end="") for j in J_prime]
print()
print(f"{'new objects j':14s} | ", end = "")
[print(f"{str(j+1):6s}  ", end="") for j in J]
print()
print("-"*14 + len(a)*8*"-")
print(f"{'c_j/a_j':14s} | ", end = "")
[print(f"{str(c_a_j):6s}  ", end="") for c_a_j in c_a]
print()
print(f"{'c_j':14s} | ", end = "")
[print(f"{str(c_j):6s}  ", end="") for c_j in c]
print()
print(f"{'a_j':14s} | ", end = "")
[print(f"{str(a_j):6s}  ", end="") for a_j in a]
print()
print(f"\nb = {b}")

old objects j` | 4       1       3       2       
new objects j  | 1       2       3       4       
----------------------------------------------
c_j/a_j        | 4       3       2       17/12   
c_j            | 96      87      72      17      
a_j            | 24      29      36      12      

b = 73


In [5]:
# initialization r = 0
r_hat = 0
R = [[".", 0]]
r = 0
cnt = 0

z_LB = -np.inf
z_globals = []

J_0 = [[]]
J_1 = [[]]

while len(R) > 0:
    # load node
    prev_r, r = R.pop(-1)
    cnt += 1

    J_r_0 = J_0[r]
    J_r_0.sort()
    J_r_1 = J_1[r]
    J_r_1.sort()

    J_r = [j for j in J if j not in J_r_0 and j not in J_r_1]

    print(f"----------------------------------------------------\n[{cnt}] r={r}({prev_r})\n")
    print(f"J_{r}_0 = {np.array(J_r_0)+1}")
    print(f"J_{r}_1 = {np.array(J_r_1)+1}")
    print(f"\nJ_{r} = {np.array(J_r)+1}")
    print()

    # build vector
    x_r_UB = np.zeros_like(J) + Fraction()
    x_r_UB[J_r_1] = 1

    # check for feasibility
    if np.sum(a*x_r_UB) > b:
        z_r_UB = np.inf

        print(f"x_r_UB = {x_r_UB}")
        print(f"z_r_UB = {z_r_UB}")

    else:
        frac_idx = 0

        for j in J_r:
            x_r_UB[j] = 1
            if a[x_r_UB==1].sum() > b:
                x_r_UB[j] = 0
                x_r_UB[j] = Fraction(b-a[x_r_UB==1].sum(), a[j])
                frac_idx = j
                break

        z_r_UB = np.sum(c*x_r_UB)

        x_r_LB = np.floor(x_r_UB)
        z_r_LB = np.sum(c*x_r_LB)

        print(f"x_r_UB = {x_r_UB} = {x_r_UB.astype(float)}")
        print(f"z_r_UB = {z_r_UB} = {int(float(z_r_UB))}+{z_r_UB-int(float(z_r_UB))} = {float(z_r_UB)}")
        print(f"x_r_LB = {x_r_LB} = {x_r_LB.astype(float)}")
        print(f"z_r_LB = {z_r_LB} = {int(float(z_r_LB))}+{z_r_LB-int(float(z_r_LB))} = {float(z_r_LB)}")

        if z_r_LB > z_LB:
            z_LB = z_r_LB
            z_globals.append((cnt, x_r_LB, z_LB))
            print("\nglobal update")

    if np.isinf(float(z_r_UB)):
        print("\npruning (infeasibility)")
        continue
    elif z_r_LB == z_r_UB:
        if z_r_LB == z_LB:
            print("\npruning (optimality)")
        else:
            print("\npruning (optimality/dominance)")
        continue
    elif z_r_UB <= z_LB:
        print("\npruning (dominance)")
        continue

    r_hat += 1
    R.append([r, r_hat])
    J_0.append(J_0[r] + [frac_idx])
    J_1.append(J_1[r])

    r_hat += 1
    R.append([r, r_hat])
    J_0.append(J_0[r])
    J_1.append(J_1[r] + [frac_idx])

    print(f"\nR = {[tmp[1] for tmp in R]}")

if len(z_globals) > 0:
    print("\n\n-------------------------\nglobals\n-------------------------")
    for idx, x, z in z_globals:
        print(f"[{idx}]: z_LB = {float(z)}")

    print(f"\noptimum found at [{idx}]: x = {z_globals[-1][1]} --> z = {int(float(z_globals[-1][2]))}+{z_globals[-1][2]-int(float(z_globals[-1][2]))} = {float(z_globals[-1][2])}.")
    print(f"selected indices J_opt = {np.sort(J_prime[x==1])+1}")
else:
    print("\nNo optimal solution found!")

----------------------------------------------------
[1] r=0(.)

J_0_0 = []
J_0_1 = []

J_0 = [1 2 3 4]

x_r_UB = [1 1 5/9 0] = [1.         1.         0.55555556 0.        ]
z_r_UB = 223 = 223+0 = 223.0
x_r_LB = [1 1 0 0] = [1. 1. 0. 0.]
z_r_LB = 183 = 183+0 = 183.0

global update

R = [1, 2]
----------------------------------------------------
[2] r=2(0)

J_2_0 = []
J_2_1 = [3]

J_2 = [1 2 4]

x_r_UB = [1 13/29 1 0] = [1.         0.44827586 1.         0.        ]
z_r_UB = 207 = 207+0 = 207.0
x_r_LB = [1 0 1 0] = [1. 0. 1. 0.]
z_r_LB = 168 = 168+0 = 168.0

R = [1, 3, 4]
----------------------------------------------------
[3] r=4(2)

J_4_0 = []
J_4_1 = [2 3]

J_4 = [1 4]

x_r_UB = [1/3 1 1 0] = [0.33333333 1.         1.         0.        ]
z_r_UB = 191 = 191+0 = 191.0
x_r_LB = [0 1 1 0] = [0. 1. 1. 0.]
z_r_LB = 159 = 159+0 = 159.0

R = [1, 3, 5, 6]
----------------------------------------------------
[4] r=6(4)

J_6_0 = []
J_6_1 = [1 2 3]

J_6 = [4]

x_r_UB = [1 1 1 0]
z_r_UB = inf

pr